# CONTENIDO

* [Introducción](#intro)
* [Descripción de datos](#descri)
* [Entrenamientos](#ml)
* [Modelo de árbol de decisión](#dt)
* [Modelo de Bosque de decisión](#bd)
* [Modelo de regresión logística](#rl)
* [Conclusión Final](#cf)

# Introducción<a id = 'intro'></a>


La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes
utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el
comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline:
Smart o Ultra.

# Descripción de datos <a id = 'descri'></a>

In [78]:
#Importación de librerías a utilizar
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [46]:
#Cargar el dataset a nuestro objero df
df = pd.read_csv('/datasets/users_behavior_upd.csv')

In [47]:
#Información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   calls        3214 non-null   float64
 1   minutes      3214 non-null   float64
 2   messages     3214 non-null   float64
 3   mb_used      3214 non-null   float64
 4   is_ultimate  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [48]:
#Mostrar una muestra de nuestro data
df.head(10)

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [49]:
df.duplicated().sum()

0

In [50]:
df.isna().sum()

calls          0
minutes        0
messages       0
mb_used        0
is_ultimate    0
dtype: int64

Nuestro dataset está conformado por 3214 filas y 5 columnas, es una dataset bastante limpio no se encuentran valores nulos, ni duplicados, la descripción de nuestras columnas es la siguiente: 

- `сalls`: Número de llamadas
- `minutes`: Duración total de la llamada en minutos
- `messages`: Número de mensajes de texto
- `mb_used`: Tráfico de Internet utilizado en MB
- `is_ultimate`: Plan para el mes actual (Ultimate - 1, Surf - 0)

# Entrenamiento <a id = 'ml'></a>

Dividiremos nuestro dataset en un conjunto de entrenamiento, uno de validación  y uno de prueba, primero definiremos nuestro objetivo y nuestras caracteristicas:

- **target:** columna 'is_ultimate'
- **features:** demás columnas

Como no contamos con otro dataset de pruebas dividiremos el nuestro a proporción 3:1:1 (60%,20%,20%). Nuestro objetivo lo estamos tratando como un valor categórico ya que estamos definiendo si pertenece a un plan o a otro, por lo que aplicaremos los siguentes métodos:

1. Árbol de decisión.
2. Bosque aleatorio.
3. Regresión logística.

In [74]:
# Adquiriendo objetivos y características
target = df['is_ultimate']
features = df.drop('is_ultimate', axis = 1)

#Dividiendo el dataset en 80% (entrenamiento y validacion)y 20% de prueba
x_features, features_test, x_target, target_test = train_test_split(features, target, test_size =.2, random_state = 12345)

#Dividiendo el dataset en entrenamiento y validacion
features_train,features_valid, target_train, target_valid = train_test_split(
    x_features, 
    x_target, 
    test_size =.25, 
    random_state = 12345)

In [67]:
#Comprobando porcentajes
print("Porcentaje de features de entrenamiento", '{:.1%}'.format(len(features_train)/len(features)))
print("Porcentaje de features de validación", '{:.1%}'.format(len(features_valid)/len(features)))
print("Porcentaje de features de pruebas", '{:.1%}'.format(len(features_test)/len(features)))

Porcentaje de features de entrenamiento 60.0%
Porcentaje de features de validación 20.0%
Porcentaje de features de pruebas 20.0%


# Módelo de árbol de decisión<a id = 'dt'></a>

In [68]:
#Ciclo for para encontrar la mejor exctitud para un arbol de decisión
best_model = None
best_accuracy  = 0

for depth in range(1,11):
    tree_model = DecisionTreeClassifier(random_state = 123456, max_depth =depth)
    tree_model.fit(features_train, target_train)
    predictions = tree_model.predict(features_valid)
    accuracy =  accuracy_score(target_valid,predictions)
    if accuracy > best_accuracy:
        best_model = tree_model
        best_accuracy = accuracy
    print('max_depth =', depth, ': ', end='')
    print(accuracy)


print("Exactitud del mejor modelo:", best_accuracy)
        

max_depth = 1 : 0.7387247278382582
max_depth = 2 : 0.7573872472783826
max_depth = 3 : 0.7651632970451011
max_depth = 4 : 0.7636080870917574
max_depth = 5 : 0.7573872472783826
max_depth = 6 : 0.7589424572317263
max_depth = 7 : 0.7713841368584758
max_depth = 8 : 0.7682737169517885
max_depth = 9 : 0.7651632970451011
max_depth = 10 : 0.7713841368584758
Exactitud del mejor modelo: 0.7713841368584758


In [89]:
#Comprobando modelo con dataset de prueba

tree_model = DecisionTreeClassifier(random_state = 123456, max_depth =7)
tree_model.fit(features_train, target_train)
predictions = tree_model.predict(features_test)
accuracy_test =  accuracy_score(target_valid,predictions)
print("Exactitud con los datos de prueba:", accuracy_test)


Exactitud con los datos de prueba: 0.6329704510108864


**Conclusión**

Nuestro modelo creado por arboles decisiones nos da una exactitud máxima de 77.13% con una profundidad de 7 árboles, de ahí nuestra exactitud empieza a bajar y en la profundidad 10 tenemos la misma exactitud que con la profundidad de 7 árboles, m{as sin embargo al utiliza nuestros datos de prueba obtuvimos una exactituda de 63.29% bajando 10%

# Módelo de bosque aleatorio <a id = 'ba'></a>

In [91]:
#Ciclo for para encontrar la mejor exctitud para un bosque aleatorio
best_est = 0
best_score  = 0

for est in range(1,11):
    forest_model =  RandomForestClassifier(random_state = 123456, n_estimators = est)
    forest_model.fit(features_train, target_train)
    score =  forest_model.score(features_valid,target_valid)
    if score > best_score:
        best_est = est
        best_score= score
    print('max_depth =', est, ': ', end='')
    print(score)


print("Exactitud del mejor modelo:", best_score)


max_depth = 1 : 0.7045101088646968
max_depth = 2 : 0.744945567651633
max_depth = 3 : 0.7433903576982893
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7807153965785381
max_depth = 6 : 0.7962674961119751
max_depth = 7 : 0.7807153965785381
max_depth = 8 : 0.7838258164852255
max_depth = 9 : 0.7807153965785381
max_depth = 10 : 0.7869362363919129
Exactitud del mejor modelo: 0.7962674961119751


In [94]:
#Comprobando modelo con dataset de prueba

forest_model = DecisionTreeClassifier(random_state = 123456, max_depth =6)
forest_model.fit(features_train, target_train)
score_test = forest_model.score(features_test,target_test)
print("Exactitud con los datos de prueba:", score_test)

Exactitud con los datos de prueba: 0.7791601866251944


**Conclusión**

Podemos observar como tenemos ligeramente mejores resultado con este modelo, nuestra exactitud mejoró a un 79.62% y a diferencia del árbol de decisión, al hacer al utilizar nuestros datos de prueba estos están en casi 78%

# Modelo de regresión logística<a id= 'rl'></a>

In [96]:
#Verificación de score con una regresión logística
rl_model = LogisticRegression(random_state=1234, solver='liblinear')
rl_model.fit(features_train, target_train)

rl_score = rl_model.score(features_valid,target_valid)
print("Exatitud de modelo de regresión logística:", rl_score )


Exatitud de modelo de regresión logística: 0.6967340590979783


In [98]:
#Comprobando exactitud del dataset de prueba

rl_score_test = rl_model.score(features_test,target_test)
print("Exatitud de modelo de regresión logística:", rl_score_test )

Exatitud de modelo de regresión logística: 0.702954898911353


**Conclusión**

Este método nos dió el resultado más bajo con un 69%, al ejecutarlo con nuestro datos de prueba aumentó un poco pero sin revasar a nuestro metodo de bosque aleatorio

# Conclusión final<a id= 'cf'></a>

La tabla la cual nos fue proporcionada ya había pasado por un proceso de preprocesamiento por lo que nuestros datos ya no contaban con valores nulos y los tipos de datos estaban acorde a la información.

Aplicamos tres modelos de entrenamiento para nuestro conjunto de datos quedando en este orden de precisión en considerando la exactitud de nuestro conjunto de pruebas:

1. Método de Bosque aleatorio
2. Regresión logística
3. Árbol de decisión

Con el método de bosque aleatorio obtuvimos una precición del 79% para nuestros datos de entrenamiento y un 77% para nuestros datos de prueba.